# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'human_nontata_promoters'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "demo_coding_vs_intergenomic_seqs"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_coding_vs_intergenomic_seqs 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

2021-10-22 23:46:07.031516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-22 23:46:07.031532: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-10-22 23:46:09.860553: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-22 23:46:09.860599: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-10-22 23:46:09.861179: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

Dataset `demo_coding_vs_intergenomic_seqs` has 2 classes: coding_seqs, intergenomic_seqs.

All lenghts of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/loc2seq/with_biopython.py:96: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


,train,test
coding_seqs,37500,12500
intergenomic_seqs,37500,12500


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 75000 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-10-22 23:46:11.532371: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 'a', 't', 'g', 'c']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


   1/1172 [..............................] - ETA: 15:20 - loss: 0.7336 - binary_accuracy: 0.4844

   5/1172 [..............................] - ETA: 14s - loss: 0.6740 - binary_accuracy: 0.5844  

  10/1172 [..............................] - ETA: 14s - loss: 0.6404 - binary_accuracy: 0.6375

  15/1172 [..............................] - ETA: 14s - loss: 0.6261 - binary_accuracy: 0.6646

...

1157/1172 [============================>.] - ETA: 0s - loss: 0.2381 - binary_accuracy: 0.9047

1161/1172 [============================>.] - ETA: 0s - loss: 0.2380 - binary_accuracy: 0.9046

1165/1172 [============================>.] - ETA: 0s - loss: 0.2380 - binary_accuracy: 0.9046

1169/1172 [============================>.] - ETA: 0s - loss: 0.2380 - binary_accuracy: 0.9046

1172/1172 [==============================] - 16s 14ms/step - loss: 0.2379 - binary_accuracy: 0.9046


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 25000 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

  1/391 [..............................] - ETA: 1:30 - loss: 0.2729 - binary_accuracy: 0.8438

 10/391 [..............................] - ETA: 2s - loss: 0.2973 - binary_accuracy: 0.8703  

 18/391 [>.............................] - ETA: 2s - loss: 0.3174 - binary_accuracy: 0.8646

 26/391 [>.............................] - ETA: 2s - loss: 0.3144 - binary_accuracy: 0.8654

 33/391 [=>............................] - ETA: 2s - loss: 0.3091 - binary_accuracy: 0.8684

...

373/391 [===========================>..] - ETA: 0s - loss: 0.3114 - binary_accuracy: 0.8699

382/391 [============================>.] - ETA: 0s - loss: 0.3109 - binary_accuracy: 0.8702

391/391 [==============================] - ETA: 0s - loss: 0.3096 - binary_accuracy: 0.8708

391/391 [==============================] - 3s 6ms/step - loss: 0.3096 - binary_accuracy: 0.8708


[0.3095621168613434, 0.8708000183105469]